# HW3 Q4 [10 pts]



## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>


All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  



<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`load_data()`

`exclude_no_pickuplocations()`

`exclude_no_tripdistance()`

`include_fare_range()`

`get_highest_tip()`

`get_total_toll()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from NYC Taxi Trips during auto-grading. You do not need to write code for unreasonable scenarios.  

Since the overall correctness of your code will require multiple function to work together correctly (i.e., all methods are interdepedent), implementing only a subset of the functions likely will lead to a low score.

### Helper functions

You are permitted to write additional helper functions, or use additional instance variables so long as the previously described functions work as required.  

#### Pyspark Imports
<span style="color:red">*Please don't modify the below cell*</span>

In [1]:
import pyspark
from pyspark.sql import SQLContext

#### Define Spark Context
<span style="color:red">*Please don't modify the below cell*</span>

In [2]:
sc
sqlContext = SQLContext(sc)

NameError: name 'sc' is not defined

### Student Section - Please compete all the functions below

#### Function to return GT Username

In [ ]:
def user():
        """
        :return: string
        your GTUsername, NOT your 9-Digit GTId  
        """         
        return 'yqi82'

#### Function to load data

In [ ]:
def load_data(gcp_storage_path):
    """
        :param gcp_storage_path: string (full gs path including file name e.g gs://bucket_name/data.csv) 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to load yellow_tripdata_2019-01.csv data from your GCP storage bucket#
    #                                                              #        
    ################################################################
    from pyspark.sql import functions
    df = sqlContext.read.option("header",True).option("inferSchema", "true").csv(gcp_storage_path)
    df = df.withColumn("trip_distance", functions.col("trip_distance").cast("float")) \
    .withColumn("fare_amount", functions.col("fare_amount").cast("float")) \
    .withColumn("tip_amount", functions.col("tip_amount").cast("float")) \
    .withColumn("tolls_amount", functions.col("tolls_amount").cast("float"))
    return df

#### Function to exclude trips that don't have a pickup location

In [ ]:
def exclude_no_pickuplocations(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to exclude trips with no pickup locations               #
    #                                                              #        
    ################################################################
    df = df.filter((df.PULocationID != 0) | (df.PULocationID.isNotNull()))
    return df

#### Function to exclude trips with no distance

In [ ]:
def exclude_no_tripdistance(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to exclude trips with no trip distances                 #
    #                                                              #        
    ################################################################
    df = df.filter((df.trip_distance != 0.0) | (df.trip_distance.isNotNull()))
    return df

#### Function to include fare amount between the range of 20 to 60 Dollars

In [ ]:
def include_fare_range(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to include trips with only within the fare range of     #
    # 20 to 60 dollars                                             #        
    ################################################################
    df = df.filter((df.fare_amount >= 20.0) & (df.fare_amount <= 60.0))
    return df

#### Function to get the highest tip amount

In [ ]:
def get_highest_tip(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: float (rounded to 2 digits)  
    """
    
    ################################################################
    # code to get the highest tip                                  #
    #                                                              #        
    ################################################################
    max_tip = df.agg({"tip_amount": "max"}).collect()[0][0]
    return round(max_tip, 2)

#### Function to get total toll amount

In [ ]:
def get_total_toll(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: float (rounded to 2 digits)  
    """
    
    ################################################################
    # code to get total toll amount                                #
    #                                                              #        
    ################################################################
    total_toll = df.agg({"tolls_amount": "sum"}).collect()[0][0]
    return round(total_toll, 2)

### Run above functions and print

#### Uncomment the cells below and test your implemented functions

#### Load data from yellow_tripdata_2019-01.csv 

In [ ]:
gcp_storage_path = "gs://yqi82/yellow_tripdata_2019-01.csv"
df = load_data(gcp_storage_path)
df.printSchema()

#### Print total numbers of rows in the dataframe

In [ ]:
df.count()

#### Print total number of rows in the dataframe after excluding trips with no pickup location

In [ ]:
df_no_pickup_locations = exclude_no_pickuplocations(df)
df_no_pickup_locations.count()

#### Print total number of rows in the dataframe after exclude trips with no distance

In [ ]:
df_no_trip_distance = exclude_no_tripdistance(df_no_pickup_locations)
df_no_trip_distance.count()

#### Print total number of rows in the dataframe after including trips with fair amount between the range of 20 to 60 Dollars

In [ ]:
df_include_fare_range = include_fare_range(df_no_trip_distance)
df_include_fare_range.count()

#### Print the highest tip amount

In [ ]:
max_tip = get_highest_tip(df_include_fare_range)
print(max_tip)

#### Print the total toll amount

In [ ]:
total_toll = get_total_toll(df_include_fare_range)
print(total_toll)